# Web Scraping of E Auction India website

In [17]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [18]:
 
headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
webpage=requests.get('https://www.eauctionsindia.com/properties/60',headers=headers).text

In [19]:
# Parsing the webpage content using BeautifulSoup with 'lxml' parser
soup=BeautifulSoup(webpage,'lxml')

In [22]:
# Extracting <div> elements with specific class attributes related to auctions
Auction=soup.find_all('div',class_='col-sm-12 col-md-4 col-lg-9')

In [23]:
# Counting the number of auction-related elements found on the webpage
len(Auction)

1

In [9]:
# This line finds the <div> element with class 'd-flex g5ere__property-detail-item', then locates the <b> tag inside it and retrieves the text.
City=soup.find('div',class_='d-flex g5ere__property-detail-item').find('b').text
print(City)

 Rajkot


In [10]:
Reserve_Price=soup.find('span',class_='text-dark').text
print(Reserve_Price)

 70,00,000


In [11]:
Category=soup.find('li',class_='me-5').find('b').text
print(Category)

Residential


In [12]:
Bank=soup.find('div',class_='card-body d-flex').find('b').text
print(Bank)

 Housing Development Finance Corporation Ltd


In [5]:
# Lists to store extracted data
Auction_ID=[]
Title=[]
State=[]
City=[]
Area_Town=[]
Address=[]
Reserve_Price=[]
Borrower_Name=[]
Category=[]
Property_type=[]
Auction_Type=[]
Auction_Start_Date=[]
Auction_End_Date=[]
Application_Submission_Date=[]
Bank=[]
Branch_Name=[]
EMD=[]

# Loop through the 'Auction' elements and extract data
for i in Auction:
# Extract Auction ID, Title, State, City, Area/Town, Address, Category, Auction Type, Reserve Price, Bank, 
# Auction Start Date, Auction End Date, Application Submission Date, Borrower Name, Property Type, EMD, Branch Name
    
    Auction_ID.append(i.find('a', class_='mb-0 icons text-primary').text)
    Title.append(i.find('h1', class_='text-primary').text)
    State.append(i.find('li', class_='col-sm-4 col-12 space').find('b').text)
    City.append(i.find('div', class_='d-flex g5ere__property-detail-item').find('b').text)
    Area_Town.append(i.select_one('.g5ere__property-detail-item strong:-soup-contains("Area/Town")').find_next_sibling('span').get_text(strip=True) if i.select_one('.g5ere__property-detail-item strong:-soup-contains("Area/Town")') else "No area found.")
    Address.append(i.select_one('.card-body .mb-4 p').get_text() if i.select_one('.card-body .mb-4 p') else "No <p> tag found.")
    Category.append(i.find('li', class_='me-5').find('b').text)
    Auction_Type.append(i.select_one('strong:-soup-contains("Auction Type") + span').get_text(strip=True) if i.select_one('strong:-soup-contains("Auction Type") + span') else "No auction type found.")
    Reserve_Price.append(i.find('span', class_='text-dark').text)
    Bank.append(i.find('div', class_='card-body d-flex').find('b').text)
    Auction_Start_Date.append(i.select_one('strong:-soup-contains("Auction Start Date") + span').get_text(strip=True) if i.select_one('strong:-soup-contains("Auction Start Date") + span') else "No auction start date found.")
    Auction_End_Date.append(i.select_one('strong:-soup-contains("Auction End Time") + span').get_text(strip=True) if i.select_one('strong:-soup-contains("Auction End Time") + span') else "No auction end date found.")
    Application_Submission_Date.append(i.select_one('strong:-soup-contains("Application Subbmision Date") + span').get_text(strip=True) if i.select_one('strong:-soup-contains("Application Subbmision Date") + span') else "No application submission date found.")
    Borrower_Name.append(i.select_one('strong:-soup-contains("Borrower Name") + span').get_text(strip=True) if i.select_one('strong:-soup-contains("Borrower Name") + span') else "No borrower name found.")
    Property_type.append(i.select_one('strong:-soup-contains("Property Type") + span').get_text(strip=True) if i.select_one('strong:-soup-contains("Property Type") + span') else "No property type found.")
    EMD.append(i.select_one('strong:-soup-contains("EMD") + span').get_text(strip=True) if i.select_one('strong:-soup-contains("EMD") + span') else "No EMD found.")
    Branch_Name.append(i.select_one('strong:-soup-contains("Branch Name") + span').get_text(strip=True) if i.select_one('strong:-soup-contains("Branch Name") + span') else "No branch name found.")

 # Creating a DataFrame to store the extracted data
df = pd.DataFrame({
    'Auction_ID': Auction_ID,
    'Title': Title,
    'State': State,
    'City': City,
    'Area_Town': Area_Town,
    'Address': Address,
    'Reserve_Price': Reserve_Price,
    'Borrower_Name':Borrower_Name,
    'Category': Category,
    'Property_type': Property_type,
    'Auction_Type': Auction_Type,
    'Auction_Start_Date': Auction_Start_Date,
    'Auction_End_Date': Auction_End_Date,
    'Application_Submission_Date': Application_Submission_Date,
    'Bank': Bank,
    'EMD':EMD,
    'Branch_Name':Branch_Name,
    
})



In [6]:
df

,Auction_ID,Title,State,City,Area_Town,Address,Reserve_Price,Borrower_Name,Category,Property_type,Auction_Type,Auction_Start_Date,Auction_End_Date,Application_Submission_Date,Bank,EMD,Branch_Name
0,60,Housing Development Finance Corporation Ltd Au...,Gujarat,Rajkot,KALAWAD ROAD,RESIDENTIAL PROPERTY KNOWN AS KRISHNA BHUMI AP...,"70,00,000",National Tyre Remould & New Tyre Junction,Residential,Plot,Sarfaesi Auction,30-08-2022 11:00 AM,30-08-2022 12:00 PM,2022-08-27 17:00:00,Housing Development Finance Corporation Ltd,"7,00,000",Rajkot


In [15]:
df.shape


(1, 17)

In [16]:
import time

final = []

start_page = 12001
end_page = 12002

for page_number in range(start_page, end_page + 1):
    url = f'https://www.eauctionsindia.com/properties/{page_number}'
    webpage = requests.get(url).text
    soup = BeautifulSoup(webpage, 'html.parser')
    Auction = soup.find_all('div', class_='col-xl-9 col-lg-8 col-md-12')
    
    for i in Auction:
        auction_data = {}
        auction_data['Auction_ID'] = i.find('a', class_='mb-0 icons text-primary').text
        auction_data['Title'] = i.find('h1', class_='text-primary').text
        auction_data['State'] = i.find('li', class_='col-sm-4 col-12 space').find('b').text
        auction_data['City'] = i.find('div', class_='d-flex g5ere__property-detail-item').find('b').text
        auction_data['Area_Town'] = (i.select_one('.g5ere__property-detail-item strong:-soup-contains("Area/Town")').find_next_sibling('span').get_text(strip=True) if i.select_one('.g5ere__property-detail-item strong:-soup-contains("Area/Town")') else "No area found.")
        auction_data['Address'] = (i.select_one('.card-body .mb-4 p').get_text() if i.select_one('.card-body .mb-4 p') else "No <p> tag found.")
        auction_data['Reserve_Price'] = i.find('span', class_='text-dark').text
        auction_data['Borrower_Name'] = (i.select_one('strong:-soup-contains("Borrower Name") + span').get_text(strip=True) if i.select_one('strong:-soup-contains("Borrower Name") + span') else "No borrower name found.")
        auction_data['Category'] = i.find('li', class_='me-5').find('b').text
        auction_data['Property_type'] = i.find_all('span')[7].text.strip()
        auction_data['Auction_Type'] = (i.select_one('strong:-soup-contains("Auction Type") + span').get_text(strip=True) if i.select_one('strong:-soup-contains("Auction Type") + span') else "No auction type found.")
        auction_data['Auction_Start_Date'] = (i.select_one('strong:-soup-contains("Auction Start Date") + span').get_text(strip=True) if i.select_one('strong:-soup-contains("Auction Start Date") + span') else "No auction start date found.")
        auction_data['Auction_End_Date'] = (i.select_one('strong:-soup-contains("Auction End Time") + span').get_text(strip=True) if i.select_one('strong:-soup-contains("Auction End Time") + span') else "No auction end date found.")
        auction_data['Application_Submission_Date'] = (i.select_one('strong:-soup-contains("Application Subbmision Date") + span').get_text(strip=True) if i.select_one('strong:-soup-contains("Application Subbmision Date") + span') else "No application submission date found.")
        auction_data['Bank'] = i.find('div', class_='card-body d-flex').find('b').text
        auction_data['EMD'] = (i.select_one('strong:-soup-contains("EMD") + span').get_text(strip=True) if i.select_one('strong:-soup-contains("EMD") + span') else "No EMD found.")
        auction_data['Branch_Name'] = (i.select_one('strong:-soup-contains("Branch Name") + span').get_text(strip=True) if i.select_one('strong:-soup-contains("Branch Name") + span') else "No branch name found.")

        final.append(auction_data)
        time.sleep(1)

df = pd.DataFrame(final)


In [17]:
df

,Auction_ID,Title,State,City,Area_Town,Address,Reserve_Price,Borrower_Name,Category,Property_type,Auction_Type,Auction_Start_Date,Auction_End_Date,Application_Submission_Date,Bank,EMD,Branch_Name
0,12001,Baroda Gujarat Gramin Bank Auctions for Reside...,Gujarat,Vadodara,Vadsar,"All the pieces & parcel of the property, Flat ...","7,29,000",Mr. Yogeshbhai Madhavbhai Solanki,Residential,Residential Flat,Sarfaesi Auction,10-10-2022 11:00 AM,10-10-2022 02:00 PM,2022-10-07 17:00:00,Baroda Gujarat Gramin Bank,"72,900",Regional Office
1,12002,Union Bank of India Auctions for Land in Phulw...,Bihar,Patna,Phulwari Sharif,Land admeasurig 8 Katha with Construction Stiu...,"1,73,60,000",M/s Manoj Kumar Rice Mills Pvt Ltd,Residential,Land,Sarfaesi Auction,21-09-2022 11:00 AM,21-09-2022 02:30 PM,2022-09-20 16:00:30,Union Bank of India,"17,36,000",Regional Office


In [18]:
df.shape

(2, 17)

In [11]:
import time
import requests
import threading
import queue

# Create a thread-safe queue for scraped data
data_queue = queue.Queue()

start_page = 136108
end_page = 139000
num_threads = 5 # Number of threads to use for parallel scraping
sleep_time = 2  # Sleep time between requests (in seconds)

# Function to scrape a range of pages
def scrape_pages(start, end):
    for page_number in range(start, end + 1):
        url = f'https://www.eauctionsindia.com/properties/{page_number}'
        webpage = requests.get(url).text # Send an HTTP request to the URL and parse the HTML content
        soup = BeautifulSoup(webpage, 'html.parser')
        Auction = soup.find_all('div', class_='col-xl-9 col-lg-8 col-md-12')

        for i in Auction:
            auction_data = {} # Construct a dictionary with the extracted data
            anchor = i.find('a', class_='mb-0 icons text-primary')
            if anchor:
                auction_data['Auction_ID'] = anchor.text
            else:
                auction_data['Auction_ID'] = "No Auction ID found"
            
            auction_data['Title'] = i.find('h1', class_='text-primary').text
            auction_data['State'] = i.find('li', class_='col-sm-4 col-12 space').find('b').text
            auction_data['City'] = i.find('div', class_='d-flex g5ere__property-detail-item').find('b').text
            auction_data['Area_Town'] = (i.select_one('.g5ere__property-detail-item strong:-soup-contains("Area/Town")').find_next_sibling('span').get_text(strip=True) if i.select_one('.g5ere__property-detail-item strong:-soup-contains("Area/Town")') else "No area found.")
            auction_data['Address'] = (i.select_one('.card-body .mb-4 p').get_text() if i.select_one('.card-body .mb-4 p') else "No <p> tag found.")
            auction_data['Reserve_Price'] = i.find('span', class_='text-dark').text
            auction_data['Borrower_Name'] = (i.select_one('strong:-soup-contains("Borrower Name") + span').get_text(strip=True) if i.select_one('strong:-soup-contains("Borrower Name") + span') else "No borrower name found.")
            auction_data['Category'] = i.find('li', class_='me-5').find('b').text
            auction_data['Property_type'] = i.find_all('span')[7].text.strip()
            auction_data['Auction_Type'] = (i.select_one('strong:-soup-contains("Auction Type") + span').get_text(strip=True) if i.select_one('strong:-soup-contains("Auction Type") + span') else "No auction type found.")
            auction_data['Auction_Start_Date'] = (i.select_one('strong:-soup-contains("Auction Start Date") + span').get_text(strip=True) if i.select_one('strong:-soup-contains("Auction Start Date") + span') else "No auction start date found.")
            auction_data['Auction_End_Date'] = (i.select_one('strong:-soup-contains("Auction End Time") + span').get_text(strip=True) if i.select_one('strong:-soup-contains("Auction End Time") + span') else "No auction end date found.")
            auction_data['Application_Submission_Date'] = (i.select_one('strong:-soup-contains("Application Submission Date") + span').get_text(strip=True) if i.select_one('strong:-soup-contains("Application Submission Date") + span') else "No application submission date found.")
            auction_data['Bank'] = i.find('div', class_='card-body d-flex').find('b').text
            auction_data['EMD'] = (i.select_one('strong:-soup-contains("EMD") + span').get_text(strip=True) if i.select_one('strong:-soup-contains("EMD") + span') else "No EMD found.")
            auction_data['Branch_Name'] = (i.select_one('strong:-soup-contains("Branch Name") + span').get_text(strip=True) if i.select_one('strong:-soup-contains("Branch Name") + span') else "No branch name found.")
            
           

             # Add the dictionary to the thread-safe queue
            data_queue.put(auction_data)
            time.sleep(sleep_time)  # Sleep time between requests

# Split the range of pages among the specified number of threads
page_range_per_thread = [(start_page + i * (end_page - start_page + 1) // num_threads,
                         start_page + (i + 1) * (end_page - start_page + 1) // num_threads - 1)
                         for i in range(num_threads)]

# Create and start threads for parallel scraping 
threads = []
for start, end in page_range_per_thread:
    thread = threading.Thread(target=scrape_pages, args=(start, end))
    threads.append(thread)
    thread.start()

# Wait for all threads to finish
for thread in threads:
    thread.join()

# Create a list to store the scraped data in order
final = []
while not data_queue.empty():
    final.append(data_queue.get())

final.sort(key=lambda x: x['Auction_ID'])  # Sort the final list by 'Auction_ID'

# Create DataFrame from the final data
df = pd.DataFrame(final)


Exception in thread Thread-12 (scrape_pages):
Traceback (most recent call last):
  File "C:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connection.py", line 203, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    raise err
  File "C:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\admin\AppData\Local\Programs\Python\Python311\Lib\site-pa

In [12]:
df

,Auction_ID,Title,State,City,Area_Town,Address,Reserve_Price,Borrower_Name,Category,Property_type,Auction_Type,Auction_Start_Date,Auction_End_Date,Application_Submission_Date,Bank,EMD,Branch_Name
0,136108,Punjab National Bank Auctions for Residential ...,Tamil Nadu,Thiruvallur,Madhavaram,"EM of Flat No 402 in 4th Floor, 99 Delights"", ...","70,00,000",Ms Sarala,Residential,Residential Flat,Sarfaesi Auction,21-09-2023 11:00 AM,21-09-2023 04:00 PM,No application submission date found.,Punjab National Bank,"7,00,000",Regional bank
1,136109,"Indian Bank Auctions for Plot in Surat, Surat",Gujarat,Surat,Surat,5.All that piece and parcel of property bearin...,"25,20,000",Mr. Dipeshbhai Premjibhai Maru,Residential,Plot,Sarfaesi Auction,20-09-2023 11:00 AM,20-09-2023 01:00 PM,No application submission date found.,Indian Bank,"2,52,000",Zonal Office
2,136110,Indian Bank Auctions for Land And Building in ...,Tamil Nadu,Thanjavur,Perumpandi,"In Thanjavur District, Kumbakonam Registration...","75,00,000",Shri.P.S. Sethuraman,Residential,Land And Building,Sarfaesi Auction,20-09-2023 10:00 AM,20-09-2023 05:00 PM,No application submission date found.,Indian Bank,"7,50,000",Regional office
3,136111,Indian Bank Auctions for Residential Flat in C...,Gujarat,Surat,Choryasi,"6.Residential Flat No.A-1/201, 2nd Floor admea...","18,90,000",M/S. Anuj Security And Man Power,Residential,Residential Flat,Sarfaesi Auction,20-09-2023 11:00 AM,20-09-2023 01:00 PM,No application submission date found.,Indian Bank,"1,89,000",Zonal Office
4,136112,Punjab National Bank Auctions for Residential ...,Tamil Nadu,Chennai,chennai,Property in the name of Mr. P. Ramesh: (Covere...,"76,00,000",Mr M Arivazhagan,Residential,Residential House,Sarfaesi Auction,21-09-2023 11:00 AM,21-09-2023 04:00 PM,No application submission date found.,Punjab National Bank,"7,60,000",Regional bank
5,136113,Indian Bank Auctions for Land And Building in ...,Tamil Nadu,Tiruppur,Veerapandi,Detailed Description of the Property: In Tirup...,"44,96,000",Mrs.R.Vijayakumari,Residential,Land And Building,Sarfaesi Auction,20-09-2023 11:00 AM,20-09-2023 05:00 PM,No application submission date found.,Indian Bank,"4,50,000",Regional office
6,136114,Indian Bank Auctions for Residential Flat in P...,Gujarat,Surat,Palsana,7.All that piece and parcels of the immovable ...,"14,25,000",Mr. Chamanial Vardilalji Prajapat,Residential,Residential Flat,Sarfaesi Auction,20-09-2023 11:00 AM,20-09-2023 01:00 PM,No application submission date found.,Indian Bank,"1,42,500",Zonal Office
7,136115,Indian Bank Auctions for Residential House in ...,Gujarat,Surat,Choryasi,8.All that piece and parcels of the immovable ...,"25,20,000",Mr.Sunilkumar Bhimrao Salave,Residential,Residential House,Sarfaesi Auction,20-09-2023 11:00 AM,20-09-2023 01:00 PM,No application submission date found.,Indian Bank,"2,52,000",Zonal Office
8,136116,ICICI Bank Auctions for Residential House in G...,Andhra Pradesh,Guntur,Guntur,Residential G+1 house built on 325 Sq Yds land...,"97,50,000",Mr. Poonthota Shankar,Residential,Residential House,Sarfaesi Auction,26-09-2023 12:00 PM,26-09-2023 01:00 PM,No application submission date found.,ICICI Bank,"9,75,000",Regional Office
9,136117,"Indian Bank Auctions for Plot in Karwad, Valsad",Gujarat,Valsad,Karwad,9.Property bearing Plot No-5 Adm-56.24 SQ. Mts...,"16,04,000",Mr. Kunna Mangraj Behera,Residential,Plot,Sarfaesi Auction,20-09-2023 11:00 AM,20-09-2023 01:00 PM,No application submission date found.,Indian Bank,"1,60,400",Zonal Office


In [13]:
df.shape

(22, 17)

In [10]:
# Specify the file name for the CSV file where the scraped data will be saved
csv_file = "scraped_data1.csv"
# Append the DataFrame 'df' to the CSV file in append mode without writing headers and with UTF-8 encoding
df.to_csv(csv_file, mode='a', index=False, header=False, encoding='utf-8')
# Print a message indicating that the data has been successfully appended to the CSV file
print(f"Data has been appended to {csv_file}")


Data has been appended to scraped_data1.csv
